In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project\\ocular-disease-recognition\\research'

In [3]:
os.getcwd()

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project\\ocular-disease-recognition\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project\\ocular-disease-recognition'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.ocularDiseaseRecognition.constants import *
from src.ocularDiseaseRecognition.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from src.ocularDiseaseRecognition import logger
from src.ocularDiseaseRecognition.utils.common import get_size
import time
from requests.exceptions import Timeout
import subprocess

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Ensure the directory exists
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

        # Use Git LFS to pull the file
        try:
            subprocess.run(['kaggle', 'datasets', 'download', '-p', os.path.dirname(self.config.local_data_file), '-d', self.config.source_URL], check=True)
            print("Dataset downloaded successfully from Kaggle.")
        except subprocess.CalledProcessError as e:
            print(f"Error downloading dataset from Kaggle: {e}")

    def extract_zip_file(self):
        # Ensure the directory exists
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        
        # Extract the zip file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            print("Zip file extracted successfully.")


In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-19 01:15:25,181: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-19 01:15:25,183: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-19 01:15:25,185: INFO: common: created directory at: artifacts]
[2024-04-19 01:15:25,187: INFO: common: created directory at: artifacts/data_ingestion]
Dataset downloaded successfully from Kaggle.
Zip file extracted successfully.
